In [81]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rank_bm25 import BM25Okapi
import time

In [82]:
#df = pd.read_csv('tweets.queries.tsv', sep='\t')
#df['tweet_lables'] = 'No Labels'
#df.to_csv('tweets.queries.tsv', sep='\t', index = False)

In [83]:
#df = pd.read_csv('data/Random Samples/5k_sample.tsv', sep='\t', header=0)
# df.to_csv('data/Random Samples/5k_sample.csv', index = False)
#df

## Reading Dataset:

In [84]:
execution_time_list = []

In [85]:
tweets_tr = pd.read_csv('data/train/tweets.queries.tsv', sep='\t', header=0, index_col=0)
tweets_tr.sort_index(inplace=True)

tweets_dev = pd.read_csv('data/dev/tweets.queries.tsv', sep='\t', header=0, index_col=0)
tweets_dev.sort_index(inplace=True)

start_time1 = time.time()
tweets_te = pd.read_csv('data/test_tweets.queries.tsv', sep='\t', header=0, index_col=0)
tweets_te.sort_index(inplace=True)
execution_time_list.append(time.time() - start_time1)

In [86]:
tweets_tr = tweets_tr.drop_duplicates("tweet_content" , keep='first')
tweets_dev = tweets_dev.drop_duplicates("tweet_content" , keep='first')
#tweets_tr

In [87]:
vclaims = pd.read_csv('data/verified_claims.docs.tsv', sep='\t', header=0, index_col=0)

#Execution Time
#vclaims = pd.read_csv('data/Random Samples/1k_sample.tsv', sep='\t', header=0, index_col=0)
#vclaims = pd.read_csv('data/Random Samples/5k_sample.tsv', sep='\t', header=0, index_col=0)
#vclaims = pd.read_csv('data/Random Samples/10k_sample.tsv', sep='\t', header=0, index_col=0)

vclaims.sort_index(inplace=True)
vclaims

,vclaim,title
0,122 detainees released from confinement at Gua...,Did 122 Prisoners Released from Guantanamo by ...
1,70 per cent of the persons arrested during pro...,70% of Arrested Charlotte Protesters Are Out-o...
2,"A ""Trump and Obama by the Numbers"" meme recoun...",Does This Meme Accurately Show ‘Trump and Obam...
3,"A ""large-scale killing"" of white farmers is ta...",Is a ‘Large-Scale Killing’ of White Farmers Un...
4,"A ""law to separate families"" was enacted prior...",Was the ‘Law to Separate Families’ Passed in 1...
...,...,...
10370,"“Slime,” a do-it-yourself gooey craft project ...",Does the “Slime” Craze Bring Serious Health Ri...
10371,“Sun tea” (tea brewed by being left to steep i...,Bacteria in Sun Tea Risk
10372,"“The Real Deal,” words of wisdom about gas, ge...",Red Thomas ‘Real Deal’ Letter
10373,“Valentine’s Day” worm.,Valentine’s Day Worm


In [88]:
#vclaims_tr = vclaims.drop_duplicates("vclaim" , keep='first')
vclaims_tr = vclaims

In [89]:
vclaims

,vclaim,title
0,122 detainees released from confinement at Gua...,Did 122 Prisoners Released from Guantanamo by ...
1,70 per cent of the persons arrested during pro...,70% of Arrested Charlotte Protesters Are Out-o...
2,"A ""Trump and Obama by the Numbers"" meme recoun...",Does This Meme Accurately Show ‘Trump and Obam...
3,"A ""large-scale killing"" of white farmers is ta...",Is a ‘Large-Scale Killing’ of White Farmers Un...
4,"A ""law to separate families"" was enacted prior...",Was the ‘Law to Separate Families’ Passed in 1...
...,...,...
10370,"“Slime,” a do-it-yourself gooey craft project ...",Does the “Slime” Craze Bring Serious Health Ri...
10371,“Sun tea” (tea brewed by being left to steep i...,Bacteria in Sun Tea Risk
10372,"“The Real Deal,” words of wisdom about gas, ge...",Red Thomas ‘Real Deal’ Letter
10373,“Valentine’s Day” worm.,Valentine’s Day Worm


In [90]:
vclaims_test = pd.read_csv('data/test/verified_claims.docs.tsv', sep='\t', header=0, index_col=0) 

#Execution Time
#vclaims_test = pd.read_csv('data/Random Samples/1k_sample.tsv', sep='\t', header=0, index_col=0)
#vclaims_test = pd.read_csv('data/Random Samples/5k_sample.tsv', sep='\t', header=0, index_col=0)
#vclaims_test = pd.read_csv('data/Random Samples/10k_sample.tsv', sep='\t', header=0, index_col=0)


vclaims_test.sort_index(inplace=True)
#vclaims_test = vclaims_test.drop_duplicates("vclaim" , keep='first')
vclaims_test

,vclaim,title
0,122 detainees released from confinement at Gua...,Did 122 Prisoners Released from Guantanamo by ...
1,70 per cent of the persons arrested during pro...,70% of Arrested Charlotte Protesters Are Out-o...
2,"A ""Trump and Obama by the Numbers"" meme recoun...",Does This Meme Accurately Show ‘Trump and Obam...
3,"A ""large-scale killing"" of white farmers is ta...",Is a ‘Large-Scale Killing’ of White Farmers Un...
4,"A ""law to separate families"" was enacted prior...",Was the ‘Law to Separate Families’ Passed in 1...
...,...,...
10370,"“Slime,” a do-it-yourself gooey craft project ...",Does the “Slime” Craze Bring Serious Health Ri...
10371,“Sun tea” (tea brewed by being left to steep i...,Bacteria in Sun Tea Risk
10372,"“The Real Deal,” words of wisdom about gas, ge...",Red Thomas ‘Real Deal’ Letter
10373,“Valentine’s Day” worm.,Valentine’s Day Worm


In [91]:
qrels_tr = pd.read_csv('data/train/tweet-vclaim-pairs.qrels', sep='\t', 
                       header=None, names=['tweet_id', '0', 'vclaim_id', 'relevance'])

qrels_dev = pd.read_csv('data/dev/tweet-vclaim-pairs.qrels', sep='\t', 
                       header=None, names=['tweet_id', '0', 'vclaim_id', 'relevance'])

In [92]:
qrels_tr

,tweet_id,0,vclaim_id,relevance
0,1,0,394,1
1,2,0,670,1
2,3,0,670,1
3,4,0,141,1
4,5,0,83,1
...,...,...,...,...
796,994,0,652,1
797,995,0,652,1
798,996,0,778,1
799,997,0,579,1


## Pre-processing Raw Text:

In [93]:
stopwords = stopwords.words('english')
url_pattern = r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))'''
username_pattern = r"@[^\s]+"
hashtag_pattern = r"\B#\w\w+"
token_pattern = r"\b[A-Za-z][A-Za-z]+\b"

In [94]:
# function of preprocessing tweet
def preprocess_tweet(tweet, 
                     url_pattern=url_pattern, username_pattern=username_pattern, 
                     hashtag_pattern=hashtag_pattern, token_pattern=token_pattern, 
                     remove_url=True, remove_username=True, remove_hashtag=True,
                     stopwords=stopwords, with_stopwordsrm=True, with_stemming=True):
    
    # remove content after '—'
    tweet = tweet.split('—')[0]
    
    # remove url
    if remove_url == True:
        tweet = re.sub(url_pattern, "", tweet)
        
    # remove @username 
    if remove_username == True:
        tweet = re.sub(username_pattern, "", tweet)
        
    # remove #hashtag
    if remove_hashtag == True:
        tweet = re.sub(hashtag_pattern, "", tweet)
    
    # lower case 
    tweet_lower = tweet.lower()
    
    # tokenization 
    words = re.findall(token_pattern, tweet_lower)
    
    # stopwords removal
    if with_stopwordsrm == True:
        words = [word for word in words if word not in stopwords]
        
    # stemming 
    if with_stemming == True:
        ps = PorterStemmer() 
        words = [ps.stem(word) for word in words]
        
    tweet_processed = " ".join(words)
    
    return tweet_processed


In [95]:
# dict tweets_tr: tweet_id -> tweet_content
tweets_tr_prep = {}
for tweet_id in tweets_tr.index:
    tweets_tr_prep[tweet_id] = preprocess_tweet(tweets_tr.loc[tweet_id, 'tweet_content'])
    

# dict tweets_dev: tweet_id -> tweet_content
tweets_dev_prep = {}
for tweet_id in tweets_dev.index:
    tweets_dev_prep[tweet_id] = preprocess_tweet(tweets_dev.loc[tweet_id, 'tweet_content'])

start_time2 = time.time()
# dict tweets_te: tweet_id -> tweet_content
tweets_te_prep = {}
for tweet_id in tweets_te.index:
    tweets_te_prep[tweet_id] = preprocess_tweet(tweets_te.loc[tweet_id, 'tweet_content'])
execution_time_list.append(time.time() - start_time2)

In [96]:
# function of preprocessing vclaim
def preprocess_text(text, token_pattern=token_pattern, stopwords=stopwords, 
                      with_stopwordsrm=True, with_stemming=True):
    # lower case 
    text_lower = text.lower()
    
    # tokenization 
    words = re.findall(token_pattern, text_lower)
    
    # stopwords removal
    if with_stopwordsrm == True:
        words = [word for word in words if word not in stopwords]
        
    # stemming 
    if with_stemming == True:
        ps = PorterStemmer() 
        words = [ps.stem(word) for word in words]
        
    text_processed = " ".join(words)
    
    return text_processed

In [97]:
# dict vclaim_prep: vlciam_id -> [vlciam_content, vclaim_title]
vclaims_prep = {}
for vclaim_id in vclaims.index:
    vclaims_prep[vclaim_id] = []
    vclaims_prep[vclaim_id].append(preprocess_text(vclaims.loc[vclaim_id, 'vclaim']))
    vclaims_prep[vclaim_id].append(preprocess_text(vclaims.loc[vclaim_id, 'title']))

In [98]:
# dict vclaim_prep: vlciam_id -> [vlciam_content, vclaim_title]
#test vclaims
vclaims_test_prep = {}
for vclaim_test_id in vclaims_test.index:
    vclaims_test_prep[vclaim_test_id] = []
    vclaims_test_prep[vclaim_test_id].append(preprocess_text(vclaims_test.loc[vclaim_test_id, 'vclaim']))
    vclaims_test_prep[vclaim_test_id].append(preprocess_text(vclaims_test.loc[vclaim_test_id, 'title']))

## Computing Cosine Similarity:

In [99]:

# function of computing consine similarity 
def compute_cs(tweets, vclaims):
    vectorizer = TfidfVectorizer()
    vclaims_tfidf = vectorizer.fit_transform(vclaims)
    
    cosine_sims = {}
    for (tweet_id, tweet_content) in tweets.items():
        tweet_tfidf = vectorizer.transform([tweet_content])
        cs = cosine_similarity(tweet_tfidf, vclaims_tfidf).flatten()
        cosine_sims[tweet_id] = cs
    
    return cosine_sims


In [100]:
# computing Cosine Similarities for train, dev and test set
vclaims_contents = [vclaim[0] for vclaim in vclaims_prep.values()]
vclaims_titles = [vclaim[1] for vclaim in vclaims_prep.values()]

vclaims_test_contents = [vclaim[0] for vclaim in vclaims_test_prep.values()]
vclaims_test_titles = [vclaim[1] for vclaim in vclaims_test_prep.values()]

cs_tvc_tr = compute_cs(tweets_tr_prep, vclaims_contents)
cs_tvt_tr = compute_cs(tweets_tr_prep, vclaims_titles)

cs_tvc_dev = compute_cs(tweets_dev_prep, vclaims_contents)
cs_tvt_dev = compute_cs(tweets_dev_prep, vclaims_titles)

start_time3 = time.time()
cs_tvc_te = compute_cs(tweets_te_prep, vclaims_test_contents)
cs_tvt_te = compute_cs(tweets_te_prep, vclaims_test_titles)
execution_time_list.append(time.time() - start_time3)

In [101]:
df_cs_tvc_tr = pd.DataFrame.from_dict(cs_tvc_tr, orient='index')
df_cs_tvt_tr = pd.DataFrame.from_dict(cs_tvt_tr, orient='index')

df_cs_tvc_dev = pd.DataFrame.from_dict(cs_tvc_dev, orient='index')
df_cs_tvt_dev = pd.DataFrame.from_dict(cs_tvt_dev, orient='index')

start_time4 = time.time()
df_cs_tvc_te = pd.DataFrame.from_dict(cs_tvc_te, orient='index')
df_cs_tvt_te = pd.DataFrame.from_dict(cs_tvt_te, orient='index')
execution_time_list.append(time.time() - start_time4)

## Computing BM25 Score:

In [102]:
# function of computing BM25 score
def compute_bm25(tokenized_tweets, tokenized_vclaims):
    bm25kapi = BM25Okapi(tokenized_vclaims)
    
    bm25s = {}
    for (tweet_id, tweet_content) in tokenized_tweets.items():
        bm25s[tweet_id] = bm25kapi.get_scores(tweet_content)
        
    return bm25s


In [103]:
# tokenizing pre-processed text
tokenized_vclaims_contents = [vclaim[0].split(" ") for vclaim in vclaims_prep.values()]
tokenized_vclaims_titles = [vclaim[1].split(" ") for vclaim in vclaims_prep.values()]

tokenized_vclaims_test_contents = [vclaim[0].split(" ") for vclaim in vclaims_test_prep.values()]
tokenized_vclaims_test_titles = [vclaim[1].split(" ") for vclaim in vclaims_test_prep.values()]

tokenized_tweets_tr = {}
for (tweet_id, tweet_content) in tweets_tr_prep.items():
    tokenized_tweets_tr[tweet_id] = tweet_content.split(" ")
    
tokenized_tweets_dev = {}
for (tweet_id, tweet_content) in tweets_dev_prep.items():
    tokenized_tweets_dev[tweet_id] = tweet_content.split(" ")

start_time5 = time.time()    
tokenized_tweets_te = {}
for (tweet_id, tweet_content) in tweets_te_prep.items():
    tokenized_tweets_te[tweet_id] = tweet_content.split(" ")
execution_time_list.append(time.time() - start_time5)

In [104]:
# computing BM25 Scores for train and test set
bm25_tvc_tr = compute_bm25(tokenized_tweets_tr, tokenized_vclaims_contents)
bm25_tvt_tr = compute_bm25(tokenized_tweets_tr, tokenized_vclaims_titles)

bm25_tvc_dev = compute_bm25(tokenized_tweets_dev, tokenized_vclaims_contents)
bm25_tvt_dev = compute_bm25(tokenized_tweets_dev, tokenized_vclaims_titles)

start_time6 = time.time()
bm25_tvc_te = compute_bm25(tokenized_tweets_te, tokenized_vclaims_test_contents)
bm25_tvt_te = compute_bm25(tokenized_tweets_te, tokenized_vclaims_test_titles)
execution_time_list.append(time.time() - start_time6)

In [105]:
df_bm25_tvc_tr = pd.DataFrame.from_dict(bm25_tvc_tr, orient='index')
df_bm25_tvt_tr = pd.DataFrame.from_dict(bm25_tvt_tr, orient='index')

df_bm25_tvc_dev = pd.DataFrame.from_dict(bm25_tvc_dev, orient='index')
df_bm25_tvt_dev = pd.DataFrame.from_dict(bm25_tvt_dev, orient='index')

start_time7 = time.time()
df_bm25_tvc_te = pd.DataFrame.from_dict(bm25_tvc_te, orient='index')
df_bm25_tvt_te = pd.DataFrame.from_dict(bm25_tvt_te, orient='index')
execution_time_list.append(time.time() - start_time7)

## Constructing dataset with 4 features and labels:

In [106]:
n=vclaims.shape[0]
data_tr_tids = []
for tweet_id in tweets_tr_prep.keys():
    for i in range(n):
        data_tr_tids.append(tweet_id)
        
# data_tr_tids = []
# for tweet_id in tweets_tr_prep.keys():
#     data_tr_tids.extend((np.ones(vclaims.shape[0]) * tweet_id).tolist())
#print(list(vclaims_prep.keys()))    
data_tr_vids = []
for i in range(tweets_tr.shape[0]):
    data_tr_vids.extend(list(vclaims_prep.keys()))
    
data_tr_labels = []
for tweet_id in tweets_tr_prep.keys():
    labels = np.zeros(vclaims.shape[0])
    #print(len(labels))
    #print(qrels_tr['tweet_id'].values)
    if tweet_id in qrels_tr['tweet_id'].values:
        #print(qrels_tr[qrels_tr['tweet_id'] == tweet_id].values)
        for index in qrels_tr[qrels_tr['tweet_id'] == tweet_id]['vclaim_id'].values:
            for claim_index, claim_value in enumerate(list(vclaims_prep.keys())):
                if index == claim_value:
                    labels[claim_index] = 1
                    
                    
                                
    data_tr_labels.extend(labels)
    
data_tr = pd.DataFrame(columns=['tid', 'vid', 'cs_tvc', 'cs_tvt', 'bm25_tvc', 'bm25_tvt', 'label'])
data_tr['tid'] = data_tr_tids
data_tr['vid'] = data_tr_vids
data_tr['cs_tvc'] = df_cs_tvc_tr.values.flatten()
data_tr['cs_tvt'] = df_cs_tvt_tr.values.flatten()
data_tr['bm25_tvc'] = df_bm25_tvc_tr.values.flatten()
data_tr['bm25_tvt'] = df_bm25_tvt_tr.values.flatten()
data_tr['label'] = data_tr_labels
data_tr

,tid,vid,cs_tvc,cs_tvt,bm25_tvc,bm25_tvt,label
0,1,0,0.000000,0.000000,0.000000,0.000000,0.0
1,1,1,0.000000,0.000000,0.000000,0.000000,0.0
2,1,2,0.033950,0.044334,4.796907,4.506260,0.0
3,1,3,0.000000,0.000000,0.000000,0.000000,0.0
4,1,4,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...
7999995,998,10370,0.000000,0.000000,0.000000,0.000000,0.0
7999996,998,10371,0.000000,0.000000,0.000000,0.000000,0.0
7999997,998,10372,0.092393,0.167614,5.400515,6.550036,0.0
7999998,998,10373,0.000000,0.000000,0.000000,0.000000,0.0


In [107]:

n=vclaims.shape[0]
data_dev_tids = []
for tweet_id in tweets_dev_prep.keys():
    for i in range(n):
        data_dev_tids.append(tweet_id)

#data_dev_tids = []
#for tweet_id in tweets_dev_prep.keys():
#    #print(tweet_id)
#    data_dev_tids.extend((np.ones(vclaims.shape[0]) * tweet_id).tolist())
    
data_dev_vids = []
for i in range(tweets_dev.shape[0]):
    data_dev_vids.extend(list(vclaims_prep.keys()))
    
data_dev_labels = []
for tweet_id in tweets_dev_prep.keys():
    labels = np.zeros(vclaims.shape[0])
    if tweet_id in qrels_dev['tweet_id'].values:
        for index in qrels_dev[qrels_dev['tweet_id'] == tweet_id]['vclaim_id'].values:
            for claim_index, claim_value in enumerate(list(vclaims_prep.keys())):
                if index == claim_value:
                    labels[claim_index] = 1
    data_dev_labels.extend(labels)
    
data_dev = pd.DataFrame(columns=['tid', 'vid', 'cs_tvc', 'cs_tvt', 'bm25_tvc', 'bm25_tvt', 'label'])
data_dev['tid'] = data_dev_tids
data_dev['vid'] = data_dev_vids
data_dev['cs_tvc'] = df_cs_tvc_dev.values.flatten()
data_dev['cs_tvt'] = df_cs_tvt_dev.values.flatten()
data_dev['bm25_tvc'] = df_bm25_tvc_dev.values.flatten()
data_dev['bm25_tvt'] = df_bm25_tvt_dev.values.flatten()
data_dev['label'] = data_dev_labels
data_dev

,tid,vid,cs_tvc,cs_tvt,bm25_tvc,bm25_tvt,label
0,0,0,0.000000,0.0,0.00000,0.0,0.0
1,0,1,0.000000,0.0,0.00000,0.0,0.0
2,0,2,0.000000,0.0,0.00000,0.0,0.0
3,0,3,0.000000,0.0,0.00000,0.0,0.0
4,0,4,0.000000,0.0,0.00000,0.0,0.0
...,...,...,...,...,...,...,...
1969995,986,10370,0.029491,0.0,3.19514,0.0,0.0
1969996,986,10371,0.000000,0.0,0.00000,0.0,0.0
1969997,986,10372,0.000000,0.0,0.00000,0.0,0.0
1969998,986,10373,0.000000,0.0,0.00000,0.0,0.0


In [108]:
#print(vclaims.shape[0])
start_time8 = time.time()
n=vclaims_test.shape[0]
#print(vclaims_test.shape[0])
data_te_tids = []
for tweet_value in tweets_te_prep.keys():
    for i in range(n):
        data_te_tids.append(tweet_value)

#print(data_te_tids)        
#data_te_tids.extend((np.ones(vclaims.shape[0]) * tweet_id).tolist())
    
data_te_vids = []
for i in range(tweets_te.shape[0]):
    data_te_vids.extend(list(vclaims_test_prep.keys()))
    
data_te = pd.DataFrame(columns=['tid', 'vid', 'cs_tvc', 'cs_tvt', 'bm25_tvc', 'bm25_tvt', 'label'])
data_te['tid'] = data_te_tids
data_te['vid'] = data_te_vids
data_te['cs_tvc'] = df_cs_tvc_te.values.flatten()
data_te['cs_tvt'] = df_cs_tvt_te.values.flatten()
data_te['bm25_tvc'] = df_bm25_tvc_te.values.flatten()
data_te['bm25_tvt'] = df_bm25_tvt_te.values.flatten()
execution_time_list.append(time.time() - start_time8)
data_te


,tid,vid,cs_tvc,cs_tvt,bm25_tvc,bm25_tvt,label
0,999,0,0.0,0.0,0.0,0.0,NaN
1,999,1,0.0,0.0,0.0,0.0,NaN
2,999,2,0.0,0.0,0.0,0.0,NaN
3,999,3,0.0,0.0,0.0,0.0,NaN
4,999,4,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...
1999995,1198,10370,0.0,0.0,0.0,0.0,NaN
1999996,1198,10371,0.0,0.0,0.0,0.0,NaN
1999997,1198,10372,0.0,0.0,0.0,0.0,NaN
1999998,1198,10373,0.0,0.0,0.0,0.0,NaN


## Linear SVC:

In [109]:
X_tr, y_tr = data_tr[['cs_tvc', 'cs_tvt', 'bm25_tvc', 'bm25_tvt']].values, data_tr['label'].values
X_dev, y_dev = data_dev[['cs_tvc', 'cs_tvt', 'bm25_tvc', 'bm25_tvt']].values, data_dev['label'].values
start_time9 = time.time()
X_te = data_te[['cs_tvc', 'cs_tvt', 'bm25_tvc', 'bm25_tvt']].values
execution_time_list.append(time.time() - start_time9)

In [110]:
# computing class weights to handle data imbalance
class_weight = {}
for label in set(y_dev):
    class_weight[label] = np.sum(y_dev == label)

In [111]:
from sklearn.svm import LinearSVC

clf = LinearSVC(C=0.1, random_state=0, class_weight=class_weight, max_iter=1000).fit(X_tr, y_tr)

C:\Users\Usama\AppData\Roaming\Python\Python38\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [112]:
# generating result for dev data set
#score_dev = np.sum(X_dev * clf.coef_, axis=1) + clf.intercept_

#result_dev_score = pd.DataFrame(columns=['tid','vid', 'score'])
#result_dev_score['tid'] = data_dev['tid']
#result_dev_score['vid'] = data_dev['vid']
#result_dev_score['score'] = score_dev

#result_dev = pd.DataFrame(columns=['tweet_id', 'Q0', 'vclaim_id', 'rank', 'score', 'tag'])

#tid = list(result_dev_score.groupby(by='tid').groups.keys())
#for i in tid:
#    idx = result_dev_score[result_dev_score['tid'] == i]['score'].idxmax()
#    inf = result_dev_score.iloc[idx]
#    result_dev = result_dev.append({'tweet_id':str(inf[0]), 'Q0':'Q0', 'vclaim_id':str(inf[1]), 
#                                    'rank':1, 'score':inf[2], 'tag': 'COVID-19'}, ignore_index=True)

In [113]:
#result_dev

In [114]:
#result_dev.to_csv('dev_set_results/golf_system_result_dev_1.csv', header=False, index=False, sep='\t')

In [115]:
# generating result for test data set
start_time10 = time.time()
score_te = np.sum(X_te * clf.coef_, axis=1) + clf.intercept_

result_te_score = pd.DataFrame(columns=['tid','vid', 'score'])
result_te_score['tid'] = data_te['tid']
result_te_score['vid'] = data_te['vid']
result_te_score['score'] = score_te
result_te = pd.DataFrame(columns=['tweet_id', 'Q0', 'vclaim_id', 'rank', 'score', 'tag'])

tid = list(result_te_score.groupby(by='tid').groups.keys())
for i in tid:
    idx = result_te_score[result_te_score['tid'] == i]['score'].idxmax()
    inf = result_te_score.iloc[idx]
    result_te = result_te.append({'tweet_id':str(inf[0]), 'Q0':'Q0', 'vclaim_id':str(inf[1]), 
                                    'rank':1, 'score':inf[2], 'tag': 'COVID-19'}, ignore_index=True)
execution_time_list.append(time.time() - start_time10)  
print(execution_time_list)
#print("--- %s seconds ---" % (time.time() - start_time))

[0.005002737045288086, 0.07403206825256348, 0.9867169857025146, 1.6071085929870605, 0.0010149478912353516, 7.268547773361206, 1.6135482788085938, 1.9235944747924805, 0.05650067329406738, 1.6026272773742676]


In [116]:
sum(execution_time_list)

15.138693809509277

In [117]:
#3.6747047901153564, 3.5895118713378906, 3.4903786182403564
#10.910948276519775, 11.244364023208618, 11.463943243026733
#14.942622900009155, 15.129878997802734, 15.138693809509277


In [118]:
result_te

,tweet_id,Q0,vclaim_id,rank,score,tag
0,999.0,Q0,6094.0,1,-0.958960,COVID-19
1,1000.0,Q0,6094.0,1,-0.974329,COVID-19
2,1001.0,Q0,582.0,1,-0.980473,COVID-19
3,1002.0,Q0,8005.0,1,-0.966464,COVID-19
4,1003.0,Q0,8005.0,1,-0.952421,COVID-19
...,...,...,...,...,...,...
195,1194.0,Q0,4693.0,1,-0.982003,COVID-19
196,1195.0,Q0,10039.0,1,-0.977978,COVID-19
197,1196.0,Q0,8855.0,1,-0.984181,COVID-19
198,1197.0,Q0,5553.0,1,-0.925080,COVID-19


In [119]:
result_te.to_csv('test_set_results/golf_system_result_te_1.csv', header=False, index=False, sep='\t')

In [120]:
#python lib/evaluate.py -s test_set_results/golf_system_result_te_1.csv -g data/data/tweet-vclaim-pairs.qrels